<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {}
}
</script>
</head>
<body>


</body>
</html>


# Parallel Analysis - Setup and Run

This notebook sets up and runs a parallel analysis on AWS.

You can learn more at [RGLab/scamp](https://github.com/RGLab/scamp)

The steps are:

* Create the code
* Configure AWS connection
* Configure the data you want to process
* Upload the data to AWS (to S3)
* Configure the processing (code to run, CPU and memory and storage to use)
* Kick off the processing


### Step A - Create the Code

1. Import the necessary libraries
2. Create the functions we'll use

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import boto3
import os.path
import sys
import datetime

In [2]:
# from https://stackoverflow.com/questions/1392413/calculating-a-directorys-size-using-python 
def get_folder_size(local_directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(local_directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / 1000000.0

In [3]:
def get_cpu_needed (analysis_size_mb):
    cpu_count = 4
    if analysis_size_mb > 100:
        cpu_count = 8
    elif analysis_size_mb > 500:
        cpu_count = 16
    elif analysis_size_mb > 1000:
        cpu_count = 32
    return cpu_count

In [4]:
def get_memory_needed (analysis_size_mb, cpu_count):
    return 2.0 * cpu_count / 4.0

In [5]:
def get_directories_and_sizes(parent_directory):
    dir_sizes = {}
    for analysis_dir in os.listdir(parent_directory):
        analysis_dir_path = os.path.join(parent_directory, analysis_dir)
        if os.path.isdir(analysis_dir_path):
            dir_sizes[analysis_dir_path] = get_folder_size(analysis_dir_path)
    return dir_sizes

In [6]:
# based on https://gist.github.com/feelinc/d1f541af4f31d09a2ec3
def upload_analysis(input_dir, s3_bucket, s3_prefix, client):
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            # construct the full local path
            local_path = os.path.join(root, filename)

            # construct the full S3 path
            relative_path = os.path.relpath(local_path, input_dir)
            s3_path = os.path.join(s3_prefix, relative_path)

            # relative_path = os.path.relpath(os.path.join(root, filename))

            print ('Searching "%s" in "%s"' % (s3_path, bucket))
            try:
                client.head_object(Bucket=s3_bucket, Key=s3_path)
                print ("Path found on S3! Skipping %s..." % s3_path)

                # try:
                    # client.delete_object(Bucket=bucket, Key=s3_path)
                # except:
                    # print "Unable to delete %s..." % s3_path
            except:
                print ("Uploading %s..." % s3_path)
                client.meta.client.upload_file(local_path, bucket, s3_path)

### Step B - Configure AWS connection

1. Configure AWS account
2. Configure AWS credentials (e.g. access key and secret key)

In [7]:
profile_widget = widgets.Text(
    value='',
    placeholder='default / sandbox',
    description='AWS Profile:',
    disabled=False
)
display(profile_widget)

Text(value='', description='AWS Profile:', placeholder='default / sandbox')

In [10]:
profile_name = profile_widget.value
print("Using '{}' profile".format(profile_name))

Using 'default' profile


In [11]:
cortex_session = boto3.session.Session(profile_name=profile_name)

### Step C - Configure the data you want to process

1. Set the input directory
2. See the list of analysis, confirm that's corrext


In [1]:
# ENTER THE NAME OF THE PARENT DIRECTORY
parent_directory = '/Users/gfinak/hackathon/hackathonData/levelData'

In [2]:
analysis_dict = get_directories_and_sizes(parent_directory)
print("The analyses to run are {}".format(list(analysis_dict.keys())))

NameError: name 'get_directories_and_sizes' is not defined

### Step D - Upload the data to AWS (to S3)

In [16]:
# ENTER THE NAME OF THE S3 BUCKET
s3_bucket = fh-hdc-cytometry-hackathon

Text(value='fh-hdc-cytometry-hackathon', description='S3 bucket:', placeholder='')

In [17]:
s3_client = cortex_session.resource('s3')
prefix = 'input'
upload_analysis(input_dir=parent_directory, s3_bucket=s3_bucket, s3_prefix=prefix, client=s3_client)

Searching "input/001.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/001.fcs/levelExprs.rds...
Searching "input/001.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/001.fcs/levelLookup.rds...
Searching "input/001.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/001.fcs/levelRes.rds...
Searching "input/002.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/002.fcs/levelExprs.rds...
Searching "input/002.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/002.fcs/levelLookup.rds...
Searching "input/002.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/002.fcs/levelRes.rds...
Searching "input/003.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/003.fcs/levelExprs.rds...
Searching "input/003.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/003.fcs/levelLookup.rds...
Searching "input/003.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading i

Searching "input/025.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/025.fcs/levelLookup.rds...
Searching "input/025.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/025.fcs/levelRes.rds...
Searching "input/026.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/026.fcs/levelExprs.rds...
Searching "input/026.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/026.fcs/levelLookup.rds...
Searching "input/026.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/026.fcs/levelRes.rds...
Searching "input/027.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/027.fcs/levelExprs.rds...
Searching "input/027.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/027.fcs/levelLookup.rds...
Searching "input/027.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/027.fcs/levelRes.rds...
Searching "input/028.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/049.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/049.fcs/levelLookup.rds...
Searching "input/049.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/049.fcs/levelRes.rds...
Searching "input/050.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/050.fcs/levelExprs.rds...
Searching "input/050.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/050.fcs/levelLookup.rds...
Searching "input/050.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/050.fcs/levelRes.rds...
Searching "input/051.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/051.fcs/levelExprs.rds...
Searching "input/051.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/051.fcs/levelLookup.rds...
Searching "input/051.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/051.fcs/levelRes.rds...
Searching "input/052.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/073.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/073.fcs/levelLookup.rds...
Searching "input/073.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/073.fcs/levelRes.rds...
Searching "input/074.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/074.fcs/levelExprs.rds...
Searching "input/074.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/074.fcs/levelLookup.rds...
Searching "input/074.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/074.fcs/levelRes.rds...
Searching "input/075.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/075.fcs/levelExprs.rds...
Searching "input/075.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/075.fcs/levelLookup.rds...
Searching "input/075.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/075.fcs/levelRes.rds...
Searching "input/076.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/097.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/097.fcs/levelLookup.rds...
Searching "input/097.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/097.fcs/levelRes.rds...
Searching "input/098.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/098.fcs/levelExprs.rds...
Searching "input/098.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/098.fcs/levelLookup.rds...
Searching "input/098.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/098.fcs/levelRes.rds...
Searching "input/099.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/099.fcs/levelExprs.rds...
Searching "input/099.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/099.fcs/levelLookup.rds...
Searching "input/099.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/099.fcs/levelRes.rds...
Searching "input/100.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

...

Searching "input/652.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/652.fcs/levelRes.rds...
Searching "input/653.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/653.fcs/levelExprs.rds...
Searching "input/653.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/653.fcs/levelLookup.rds...
Searching "input/653.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/653.fcs/levelRes.rds...
Searching "input/654.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/654.fcs/levelExprs.rds...
Searching "input/654.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/654.fcs/levelLookup.rds...
Searching "input/654.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/654.fcs/levelRes.rds...
Searching "input/655.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/655.fcs/levelExprs.rds...
Searching "input/655.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading inpu

Searching "input/677.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/677.fcs/levelLookup.rds...
Searching "input/677.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/677.fcs/levelRes.rds...
Searching "input/678.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/678.fcs/levelExprs.rds...
Searching "input/678.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/678.fcs/levelLookup.rds...
Searching "input/678.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/678.fcs/levelRes.rds...
Searching "input/679.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/679.fcs/levelExprs.rds...
Searching "input/679.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/679.fcs/levelLookup.rds...
Searching "input/679.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/679.fcs/levelRes.rds...
Searching "input/680.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/701.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/701.fcs/levelLookup.rds...
Searching "input/701.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/701.fcs/levelRes.rds...
Searching "input/702.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/702.fcs/levelExprs.rds...
Searching "input/702.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/702.fcs/levelLookup.rds...
Searching "input/702.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/702.fcs/levelRes.rds...
Searching "input/703.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/703.fcs/levelExprs.rds...
Searching "input/703.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/703.fcs/levelLookup.rds...
Searching "input/703.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/703.fcs/levelRes.rds...
Searching "input/704.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/725.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/725.fcs/levelLookup.rds...
Searching "input/725.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/725.fcs/levelRes.rds...
Searching "input/726.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/726.fcs/levelExprs.rds...
Searching "input/726.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/726.fcs/levelLookup.rds...
Searching "input/726.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/726.fcs/levelRes.rds...
Searching "input/727.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/727.fcs/levelExprs.rds...
Searching "input/727.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/727.fcs/levelLookup.rds...
Searching "input/727.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/727.fcs/levelRes.rds...
Searching "input/728.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

Searching "input/749.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/749.fcs/levelLookup.rds...
Searching "input/749.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/749.fcs/levelRes.rds...
Searching "input/750.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/750.fcs/levelExprs.rds...
Searching "input/750.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/750.fcs/levelLookup.rds...
Searching "input/750.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/750.fcs/levelRes.rds...
Searching "input/751.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/751.fcs/levelExprs.rds...
Searching "input/751.fcs/levelLookup.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/751.fcs/levelLookup.rds...
Searching "input/751.fcs/levelRes.rds" in "fh-hdc-cytometry-hackathon"
Uploading input/751.fcs/levelRes.rds...
Searching "input/752.fcs/levelExprs.rds" in "fh-hdc-cytometry-hackathon"
Uploading inp

### Step E - Configure the processing

1. Set the code location (GitHub repo)
2. Set the startup command
3. Configure CPU and memory for each analysis
4. Configure storage for each analysis

#### Steps E1 and E2 - set GitHub repo and analysis command

#### Step E3 - Configure CPU and memory for each analysis

In [ ]:
# get info for analysis
analysis_info = {}
for analysis_dir in os.listdir(parent_directory):
    analysis_dir_path = os.path.join(parent_directory, analysis_dir)
    if os.path.isdir(analysis_dir_path):
        folder_size_mb = get_folder_size(analysis_dir_path)
        cpu_count_needed = get_cpu_needed(folder_size_mb)
        mem_gb_needed = get_memory_needed(folder_size_mb, cpu_count_needed)
        print ("Analysis {} is {} MB and needs {} CPU and {} GB of RAM".format(analysis_dir_path,folder_size_mb, cpu_count_needed, mem_gb_needed))
        analysis_params = {"size_mb": folder_size_mb, "cpu_count": cpu_count_needed, "mem_gb": mem_gb_needed}
        analysis_info[analysis_dir_path] = analysis_params
print (analysis_info)

In [ ]:
analysis_params = {} 
# key is the analysis folder name
# value is a dict of input size, CPU info, memory info, storage info

### Step F - Kick off the processing

1. Name the batch
   * Add the ability to email somene when it's done?
2. Kick off the analysis (start the batch)
3. Confirm it has started correctly

#### Step F1 - Name the batch

In [42]:
cortex = boto3.session.Session(profile_name='hackathon')
batch_client = cortex.client('batch', region_name='us-west-2')

In [45]:
def create_batch_job(r_script_s3_url
                     , s3_input_dir
                     , s3_output_dir
                     , cpu_count
                     , mem_mb
                     , job_name
                     , job_queue
                     , job_definition
                     , client
                     , batch_file_s3_url = 's3://fh-hdc-cytometry-hackathon/startup.sh'):
    response = client.submit_job(
        jobName = job_name
        ,jobQueue = job_queue
        ,jobDefinition=job_definition
        ,containerOverrides={
            'vcpus': cpu_count,
            'memory': mem_mb,
            'command': [
                'startup.sh',
            ],
            "environment": [ 
             { 
                "name": "BATCH_FILE_TYPE",
                "value": "script"
             },
             { 
                "name": "BATCH_FILE_S3_URL",
                "value": batch_file_s3_url
             },
             { 
                "name": "R_SCRIPT_S3_URL",
                "value": r_script_s3_url
             },
             { 
                "name": "S3_INPUT_DIR",
                "value": s3_input_dir
             },
             { 
                "name": "S3_OUTPUT_DIR",
                "value": s3_output_dir
             }
            ]
        }
        )
    return response

In [48]:
job_queue = 'dnambi-hackathon-queue'
job_definition = 'fetch_and_run_1:5'
job_name = 'dnambi-boto3-test-job-2'
r_script_s3_url = 's3://fh-hdc-cytometry-hackathon/r-code/analysis.r'
s3_input_dir = 's3://fh-hdc-cytometry-hackathon/input'
s3_output_dir = 's3://fh-hdc-cytometry-hackathon/output'
cpu_count = 16
mem_mb = 32000


response = create_batch_job (r_script_s3_url = r_script_s3_url
                     , s3_input_dir = s3_input_dir
                     , s3_output_dir = s3_output_dir
                     , cpu_count = cpu_count
                     , mem_mb = mem_mb
                     , job_name = job_name
                     , job_queue = job_queue
                     , job_definition = job_definition
                     , client = batch_client)

In [49]:
print (response)

{'ResponseMetadata': {'RequestId': '9ed24ef6-fa3a-11e8-a8ec-ab5a643eca6f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 07 Dec 2018 16:10:31 GMT', 'content-type': 'application/json', 'content-length': '84', 'connection': 'keep-alive', 'x-amzn-requestid': '9ed24ef6-fa3a-11e8-a8ec-ab5a643eca6f', 'x-amz-apigw-id': 'Ri06qHNyvHcFS3g=', 'x-amzn-trace-id': 'Root=1-5c0a9b77-c3a675ca30505e0e7b94929f;Sampled=0'}, 'RetryAttempts': 0}, 'jobName': 'dnambi-boto3-test-job-2', 'jobId': 'bdf51e4e-f621-46be-8025-c0336e471845'}


#### Step F2 - Kick off the analysis processing

#### Step F3 - Confirm the processing has started